# Learn OpenAI Whisper - Chapter 8

## Diarizing speech with WhisperX and NVIDIA's NeMo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1-TSAhmGl5QplD9-NzWLOONkH43SEMwdF)

This notebook offers a step-by-step guide to implementing speaker diarization using [WhisperX](https://replicate.com/dinozoiddev/whisperx) and NVIDIA's NeMo. WhisperX combines the power of `whisper-large-v3` with diarization upgrades, while NeMo provides a robust framework for speaker diarization. The goal is to demonstrate how these tools can be integrated to accurately identify "who spoke when" in an audio file, enhancing the capabilities of automated speech recognition (ASR) systems.

### Notebook Structure

The notebook is organized into key sections, each addressing a specific aspect of the diarization process:

1. **Setting up the environment**: Install necessary dependencies, including Whisper, NeMo, and supporting libraries.

2. **Streamlining the diarization workflow with helper functions**: Utility functions for audio manipulation, data processing, and speaker embedding generation.

3. **Separating music from speech using Demucs**: Separate music from speech using Demucs and convert audio to mono format for NeMo compatibility.

4. **Transcribing audio using WhisperX**: Transcribe audio using WhisperX, leveraging its high accuracy in speech recognition.

5. **Aligning Transcriptions with Wav2Vec2**: Align the transcription with the original audio using Wav2Vec2, a neural network designed for speech recognition and alignment tasks.

6. **Using NeMo's MSDD model for speaker diarization**: Employ NVIDIA's NeMo MSDD model to separate transcribed audio into segments based on speaker identity.

7. **Mapping speakers to sentences according to timestamps**: Map identified speakers to corresponding sentences in the transcription based on timestamps.

8. **Enhancing speaker attribution with punctuation-based realignment**: Refine speaker labels using punctuation, addressing scenarios where sentences are split between speakers or background comments occur.

9. **Finalizing the diarization process**: : Cleanup, result export, and speaker name mapping.

This notebook provides a comprehensive resource for researchers, developers, and practitioners interested in exploring advanced diarization techniques within ASR systems. By integrating Whisper's transcription capabilities with NeMo's diarization framework, it offers a powerful solution for analyzing speech in audio recordings.

# Setting up the environment

This section focuses on installing the necessary Python libraries and tools for the diarization process:

- **whisperX**: An enhanced extension of OpenAI's Whisper model, providing additional functionality.
- **nemo_toolkit[asr]**: NVIDIA's NeMo toolkit for automatic speech recognition, serving as the backbone for speaker diarization.
- **demucs**: A library for music source separation, enabling the isolation of speech from background music during audio preprocessing.
- **dora-search, lameenc, and openunmix**: Tools and libraries for audio processing, improving the quality and compatibility of audio data for diarization tasks.
- **deepmultilingualpunctuation**: A library for adding punctuation to transcriptions, enhancing readability and structure.
- **wget and pydub**: Utilities for downloading and manipulating audio files, simplifying audio data handling within the Python environment.

These libraries work together to form a comprehensive toolset for processing audio files, transcribing speech, and performing speaker diarization. Each tool contributes to a specific aspect of the process, from audio data preparation to accurate transcription and speaker identification.

In [ ]:
!pip install -q git+https://github.com/m-bain/whisperX.git@78dcfaab51005aa703ee21375f81ed31bc248560
!pip install -q --no-build-isolation nemo_toolkit[asr]==1.22.0
!pip install -q --no-deps git+https://github.com/facebookresearch/demucs#egg=demucs
!pip install -q dora-search "lameenc>=1.2" openunmix
!pip install -q deepmultilingualpunctuation
!pip install -q wget pydub

# RESTART
![Restart_the_runtime_600x102.png](https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter08/Restart_the_runtime_600x102.png)

In [ ]:
import os
import wget
from omegaconf import OmegaConf
import json
import shutil
from faster_whisper import WhisperModel
import whisperx
import torch
from pydub import AudioSegment
from nemo.collections.asr.models.msdd_models import NeuralDiarizer
from deepmultilingualpunctuation import PunctuationModel
import re
import logging
import nltk
from whisperx.alignment import DEFAULT_ALIGN_MODELS_HF, DEFAULT_ALIGN_MODELS_TORCH
from whisperx.utils import LANGUAGES, TO_LANGUAGE_CODE

In [ ]:
# Download sample multi-speaker audio file
# Source: www.youtube.com/@Channel4News
!wget -nv https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter08/20150415-Fracking_the_debate.mp4

# Name of the audio file
audio_path = "/content/20150415-Fracking_the_debate.mp4"

# Whether to enable music removal from speech, helps increase diarization quality but uses alot of ram
enable_stemming = True

# (choose from 'tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large')
whisper_model_name = "large-v2"

# replaces numerical digits with their pronounciation, increases diarization accuracy
suppress_numerals = True

batch_size = 8

language = None  # autodetect language

device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Streamlining the diarization workflow with helper functions

This section introduces a set of helper functions designed to streamline the process of diarizing speech using Whisper and NeMo. These functions play a crucial role in managing audio data, aligning transcriptions with speaker identities, and enhancing the overall workflow. Here's a brief overview of the key functions:

- **`create_config()`**: Sets up essential parameters for the diarization process by initializing and returning a configuration object.

- **`get_word_ts_anchor()`**: Determines the anchor timestamp for words, enabling accurate alignment between spoken words and their timestamps in the audio.

- **`get_words_speaker_mapping()`**: Maps each word in the transcription to the corresponding speaker based on the diarization results, ensuring accurate speaker attribution.

- **`get_first_word_idx_of_sentence()`** and **`get_last_word_idx_of_sentence()`**: Identifies the indices of the first and last words in a sentence, facilitating sentence-level processing for speaker attribution and alignment.

- **`get_realigned_ws_mapping_with_punctuation()`**: Enhances the accuracy of speaker attribution by adjusting the word-to-speaker mapping considering punctuation, particularly in complex conversational scenarios.

- **`get_sentences_speaker_mapping()`**: Generates a mapping of entire sentences to speakers, providing a higher-level view of speaker contributions throughout the audio.

- **`get_speaker_aware_transcript()`**: Produces a transcript that integrates textual content with speaker information, creating a cohesive and speaker-aware format.

- **`format_timestamp()`** and **`write_srt()`**: Converts timestamps into a human-readable format and outputs the diarization results in SubRip Text (SRT) format for subtitles or detailed analysis.

- **`find_numeral_symbol_tokens()`**, **`_get_next_start_timestamp()`**, and **`filter_missing_timestamps()`**: Assist in processing numerical data, ensuring continuity in timestamp sequences, and maintaining the integrity of temporal information.

- **`cleanup()`**, **`process_language_arg()`**, **`transcribe()`**, and **`transcribe_batched()`**: Handle temporary file cleanup, language argument processing, audio transcription using Whisper, and batch processing for efficient transcription.

These functions collectively form the backbone of the diarization workflow, enabling seamless integration of Whisper's transcription capabilities with NeMo's advanced diarization features.

In [ ]:
punct_model_langs = [
    "en",
    "fr",
    "de",
    "es",
    "it",
    "nl",
    "pt",
    "bg",
    "pl",
    "cs",
    "sk",
    "sl",
]
wav2vec2_langs = list(DEFAULT_ALIGN_MODELS_TORCH.keys()) + list(
    DEFAULT_ALIGN_MODELS_HF.keys()
)

whisper_langs = sorted(LANGUAGES.keys()) + sorted(
    [k.title() for k in TO_LANGUAGE_CODE.keys()]
)


def create_config(output_dir):
    DOMAIN_TYPE = "telephonic"  # Can be meeting, telephonic, or general based on domain type of the audio file
    CONFIG_FILE_NAME = f"diar_infer_{DOMAIN_TYPE}.yaml"
    CONFIG_URL = f"https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/{CONFIG_FILE_NAME}"
    MODEL_CONFIG = os.path.join(output_dir, CONFIG_FILE_NAME)
    if not os.path.exists(MODEL_CONFIG):
        MODEL_CONFIG = wget.download(CONFIG_URL, output_dir)

    config = OmegaConf.load(MODEL_CONFIG)

    data_dir = os.path.join(output_dir, "data")
    os.makedirs(data_dir, exist_ok=True)

    meta = {
        "audio_filepath": os.path.join(output_dir, "mono_file.wav"),
        "offset": 0,
        "duration": None,
        "label": "infer",
        "text": "-",
        "rttm_filepath": None,
        "uem_filepath": None,
    }
    with open(os.path.join(data_dir, "input_manifest.json"), "w") as fp:
        json.dump(meta, fp)
        fp.write("\n")

    pretrained_vad = "vad_multilingual_marblenet"
    pretrained_speaker_model = "titanet_large"
    config.num_workers = 0  # Workaround for multiprocessing hanging with ipython issue
    config.diarizer.manifest_filepath = os.path.join(data_dir, "input_manifest.json")
    config.diarizer.out_dir = (
        output_dir  # Directory to store intermediate files and prediction outputs
    )

    config.diarizer.speaker_embeddings.model_path = pretrained_speaker_model
    config.diarizer.oracle_vad = (
        False  # compute VAD provided with model_path to vad config
    )
    config.diarizer.clustering.parameters.oracle_num_speakers = False

    # Here, we use our in-house pretrained NeMo VAD model
    config.diarizer.vad.model_path = pretrained_vad
    config.diarizer.vad.parameters.onset = 0.8
    config.diarizer.vad.parameters.offset = 0.6
    config.diarizer.vad.parameters.pad_offset = -0.05
    config.diarizer.msdd_model.model_path = (
        "diar_msdd_telephonic"  # Telephonic speaker diarization model
    )

    return config


def get_word_ts_anchor(s, e, option="start"):
    if option == "end":
        return e
    elif option == "mid":
        return (s + e) / 2
    return s


def get_words_speaker_mapping(wrd_ts, spk_ts, word_anchor_option="start"):
    s, e, sp = spk_ts[0]
    wrd_pos, turn_idx = 0, 0
    wrd_spk_mapping = []
    for wrd_dict in wrd_ts:
        ws, we, wrd = (
            int(wrd_dict["start"] * 1000),
            int(wrd_dict["end"] * 1000),
            wrd_dict["word"],
        )
        wrd_pos = get_word_ts_anchor(ws, we, word_anchor_option)
        while wrd_pos > float(e):
            turn_idx += 1
            turn_idx = min(turn_idx, len(spk_ts) - 1)
            s, e, sp = spk_ts[turn_idx]
            if turn_idx == len(spk_ts) - 1:
                e = get_word_ts_anchor(ws, we, option="end")
        wrd_spk_mapping.append(
            {"word": wrd, "start_time": ws, "end_time": we, "speaker": sp}
        )
    return wrd_spk_mapping


sentence_ending_punctuations = ".?!"


def get_first_word_idx_of_sentence(word_idx, word_list, speaker_list, max_words):
    is_word_sentence_end = (
        lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
    )
    left_idx = word_idx
    while (
        left_idx > 0
        and word_idx - left_idx < max_words
        and speaker_list[left_idx - 1] == speaker_list[left_idx]
        and not is_word_sentence_end(left_idx - 1)
    ):
        left_idx -= 1

    return left_idx if left_idx == 0 or is_word_sentence_end(left_idx - 1) else -1


def get_last_word_idx_of_sentence(word_idx, word_list, max_words):
    is_word_sentence_end = (
        lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
    )
    right_idx = word_idx
    while (
        right_idx < len(word_list)
        and right_idx - word_idx < max_words
        and not is_word_sentence_end(right_idx)
    ):
        right_idx += 1

    return (
        right_idx
        if right_idx == len(word_list) - 1 or is_word_sentence_end(right_idx)
        else -1
    )


def get_realigned_ws_mapping_with_punctuation(
    word_speaker_mapping, max_words_in_sentence=50
):
    is_word_sentence_end = (
        lambda x: x >= 0
        and word_speaker_mapping[x]["word"][-1] in sentence_ending_punctuations
    )
    wsp_len = len(word_speaker_mapping)

    words_list, speaker_list = [], []
    for k, line_dict in enumerate(word_speaker_mapping):
        word, speaker = line_dict["word"], line_dict["speaker"]
        words_list.append(word)
        speaker_list.append(speaker)

    k = 0
    while k < len(word_speaker_mapping):
        line_dict = word_speaker_mapping[k]
        if (
            k < wsp_len - 1
            and speaker_list[k] != speaker_list[k + 1]
            and not is_word_sentence_end(k)
        ):
            left_idx = get_first_word_idx_of_sentence(
                k, words_list, speaker_list, max_words_in_sentence
            )
            right_idx = (
                get_last_word_idx_of_sentence(
                    k, words_list, max_words_in_sentence - k + left_idx - 1
                )
                if left_idx > -1
                else -1
            )
            if min(left_idx, right_idx) == -1:
                k += 1
                continue

            spk_labels = speaker_list[left_idx : right_idx + 1]
            mod_speaker = max(set(spk_labels), key=spk_labels.count)
            if spk_labels.count(mod_speaker) < len(spk_labels) // 2:
                k += 1
                continue

            speaker_list[left_idx : right_idx + 1] = [mod_speaker] * (
                right_idx - left_idx + 1
            )
            k = right_idx

        k += 1

    k, realigned_list = 0, []
    while k < len(word_speaker_mapping):
        line_dict = word_speaker_mapping[k].copy()
        line_dict["speaker"] = speaker_list[k]
        realigned_list.append(line_dict)
        k += 1

    return realigned_list


def get_sentences_speaker_mapping(word_speaker_mapping, spk_ts):
    sentence_checker = nltk.tokenize.PunktSentenceTokenizer().text_contains_sentbreak
    s, e, spk = spk_ts[0]
    prev_spk = spk

    snts = []
    snt = {"speaker": f"Speaker {spk}", "start_time": s, "end_time": e, "text": ""}

    for wrd_dict in word_speaker_mapping:
        wrd, spk = wrd_dict["word"], wrd_dict["speaker"]
        s, e = wrd_dict["start_time"], wrd_dict["end_time"]
        if spk != prev_spk or sentence_checker(snt["text"] + " " + wrd):
            snts.append(snt)
            snt = {
                "speaker": f"Speaker {spk}",
                "start_time": s,
                "end_time": e,
                "text": "",
            }
        else:
            snt["end_time"] = e
        snt["text"] += wrd + " "
        prev_spk = spk

    snts.append(snt)
    return snts


def get_speaker_aware_transcript(sentences_speaker_mapping, f):
    previous_speaker = sentences_speaker_mapping[0]["speaker"]
    f.write(f"{previous_speaker}: ")

    for sentence_dict in sentences_speaker_mapping:
        speaker = sentence_dict["speaker"]
        sentence = sentence_dict["text"]

        # If this speaker doesn't match the previous one, start a new paragraph
        if speaker != previous_speaker:
            f.write(f"\n\n{speaker}: ")
            previous_speaker = speaker

        # No matter what, write the current sentence
        f.write(sentence + " ")


def format_timestamp(
    milliseconds: float, always_include_hours: bool = False, decimal_marker: str = "."
):
    assert milliseconds >= 0, "non-negative timestamp expected"

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
    return (
        f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
    )


def write_srt(transcript, file):
    """
    Write a transcript to a file in SRT format.

    """
    for i, segment in enumerate(transcript, start=1):
        # write srt lines
        print(
            f"{i}\n"
            f"{format_timestamp(segment['start_time'], always_include_hours=True, decimal_marker=',')} --> "
            f"{format_timestamp(segment['end_time'], always_include_hours=True, decimal_marker=',')}\n"
            f"{segment['speaker']}: {segment['text'].strip().replace('-->', '->')}\n",
            file=file,
            flush=True,
        )


def find_numeral_symbol_tokens(tokenizer):
    numeral_symbol_tokens = [
        -1,
    ]
    for token, token_id in tokenizer.get_vocab().items():
        has_numeral_symbol = any(c in "0123456789%$£" for c in token)
        if has_numeral_symbol:
            numeral_symbol_tokens.append(token_id)
    return numeral_symbol_tokens


def _get_next_start_timestamp(word_timestamps, current_word_index, final_timestamp):
    # if current word is the last word
    if current_word_index == len(word_timestamps) - 1:
        return word_timestamps[current_word_index]["start"]

    next_word_index = current_word_index + 1
    while current_word_index < len(word_timestamps) - 1:
        if word_timestamps[next_word_index].get("start") is None:
            # if next word doesn't have a start timestamp
            # merge it with the current word and delete it
            word_timestamps[current_word_index]["word"] += (
                " " + word_timestamps[next_word_index]["word"]
            )

            word_timestamps[next_word_index]["word"] = None
            next_word_index += 1
            if next_word_index == len(word_timestamps):
                return final_timestamp

        else:
            return word_timestamps[next_word_index]["start"]


def filter_missing_timestamps(
    word_timestamps, initial_timestamp=0, final_timestamp=None
):
    # handle the first and last word
    if word_timestamps[0].get("start") is None:
        word_timestamps[0]["start"] = (
            initial_timestamp if initial_timestamp is not None else 0
        )
        word_timestamps[0]["end"] = _get_next_start_timestamp(
            word_timestamps, 0, final_timestamp
        )

    result = [
        word_timestamps[0],
    ]

    for i, ws in enumerate(word_timestamps[1:], start=1):
        # if ws doesn't have a start and end
        # use the previous end as start and next start as end
        if ws.get("start") is None and ws.get("word") is not None:
            ws["start"] = word_timestamps[i - 1]["end"]
            ws["end"] = _get_next_start_timestamp(word_timestamps, i, final_timestamp)

        if ws["word"] is not None:
            result.append(ws)
    return result


def cleanup(path: str):
    """path could either be relative or absolute."""
    # check if file or directory exists
    if os.path.isfile(path) or os.path.islink(path):
        # remove file
        os.remove(path)
    elif os.path.isdir(path):
        # remove directory and all its content
        shutil.rmtree(path)
    else:
        raise ValueError("Path {} is not a file or dir.".format(path))


def process_language_arg(language: str, model_name: str):
    """
    Process the language argument to make sure it's valid and convert language names to language codes.
    """
    if language is not None:
        language = language.lower()
    if language not in LANGUAGES:
        if language in TO_LANGUAGE_CODE:
            language = TO_LANGUAGE_CODE[language]
        else:
            raise ValueError(f"Unsupported language: {language}")

    if model_name.endswith(".en") and language != "en":
        if language is not None:
            logging.warning(
                f"{model_name} is an English-only model but received '{language}'; using English instead."
            )
        language = "en"
    return language


def transcribe(
    audio_file: str,
    language: str,
    model_name: str,
    compute_dtype: str,
    suppress_numerals: bool,
    device: str,
):
    from faster_whisper import WhisperModel
    from helpers import find_numeral_symbol_tokens, wav2vec2_langs

    # Faster Whisper non-batched
    # Run on GPU with FP16
    whisper_model = WhisperModel(model_name, device=device, compute_type=compute_dtype)

    # or run on GPU with INT8
    # model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
    # or run on CPU with INT8
    # model = WhisperModel(model_size, device="cpu", compute_type="int8")

    if suppress_numerals:
        numeral_symbol_tokens = find_numeral_symbol_tokens(whisper_model.hf_tokenizer)
    else:
        numeral_symbol_tokens = None

    if language is not None and language in wav2vec2_langs:
        word_timestamps = False
    else:
        word_timestamps = True

    segments, info = whisper_model.transcribe(
        audio_file,
        language=language,
        beam_size=5,
        word_timestamps=word_timestamps,  # TODO: disable this if the language is supported by wav2vec2
        suppress_tokens=numeral_symbol_tokens,
        vad_filter=True,
    )
    whisper_results = []
    for segment in segments:
        whisper_results.append(segment._asdict())
    # clear gpu vram
    del whisper_model
    torch.cuda.empty_cache()
    return whisper_results, language


def transcribe_batched(
    audio_file: str,
    language: str,
    batch_size: int,
    model_name: str,
    compute_dtype: str,
    suppress_numerals: bool,
    device: str,
):
    import whisperx

    # Faster Whisper batched
    whisper_model = whisperx.load_model(
        model_name,
        device,
        compute_type=compute_dtype,
        asr_options={"suppress_numerals": suppress_numerals},
    )
    audio = whisperx.load_audio(audio_file)
    result = whisper_model.transcribe(audio, language=language, batch_size=batch_size)
    del whisper_model
    torch.cuda.empty_cache()
    return result["segments"], result["language"]

# 3. Separating music from speech using Demucs

This section focuses on a crucial preprocessing step that enhances the clarity of speech before diarization. Demucs, a deep learning model for music source separation, is introduced as a powerful tool for isolating vocals from complex audio tracks.

Separating music from speech is essential when dealing with recordings containing background music or non-speech elements. By extracting the vocal component, the diarization system can more effectively analyze and attribute speech to the correct speakers, as the spectral and temporal characteristics of their speech signals become more pronounced and less obscured by music.

Demucs employs a neural network trained to distinguish between different audio sources within a mixture. When applied to an audio file, it separates the vocal track from the instrumental, allowing subsequent tools like Whisper and NeMo to process the speech without interference from background music.

This separation step not only benefits the accuracy of speaker diarization but also improves the performance of downstream tasks that require clean speech input, such as transcription and speech recognition. By incorporating Demucs into the preprocessing pipeline, the notebook ensures that the input to the diarization system is optimized for the best possible results.

In [ ]:
if enable_stemming:
    # Isolate vocals from the rest of the audio

    return_code = os.system(
        f'python3 -m demucs.separate -n htdemucs --two-stems=vocals "{audio_path}" -o "temp_outputs"'
    )

    if return_code != 0:
        logging.warning("Source splitting failed, using original audio file.")
        vocal_target = audio_path
    else:
        vocal_target = os.path.join(
            "temp_outputs",
            "htdemucs",
            os.path.splitext(os.path.basename(audio_path))[0],
            "vocals.wav",
        )
else:
    vocal_target = audio_path

# 4. Transcribing audio using WhisperX

This section focuses on leveraging WhisperX to accurately transcribe audio content. Whisper's robust capabilities enable it to convert speech to text across a wide range of languages and dialects.

The transcription process involves processing the audio file through WhisperX to generate a set of text segments, each accompanied by timestamps indicating when the segment was spoken. This step lays the foundation for speaker diarization and further analysis by providing the necessary textual content..


In [ ]:
compute_type = "float16"
# or run on GPU with INT8
# compute_type = "int8_float16"
# or run on CPU with INT8
# compute_type = "int8"

if batch_size != 0:
    whisper_results, language = transcribe_batched(
        vocal_target,
        language,
        batch_size,
        whisper_model_name,
        compute_type,
        suppress_numerals,
        device,
    )
else:
    whisper_results, language = transcribe(
        vocal_target,
        language,
        whisper_model_name,
        compute_type,
        suppress_numerals,
        device,
    )

# 5. Aligning the transcription with the original audio using Wav2Vec2

Wav2Vec2, a large-scale neural network, is employed here to learn speech representations that are useful for various speech processing tasks, including speech recognition and alignment.

The code loads the Wav2Vec2 alignment model and uses it to align the transcription segments with the original audio signal contained in the `vocal_target` file. This process involves finding the exact timestamps in the audio signal where each segment was spoken and aligning the text accordingly.

By combining the outputs of Whisper and Wav2Vec2, the code produces a fully aligned transcription of the speech contained in the `vocal_target` file. This aligned transcription is valuable for downstream tasks such as speaker diarization, sentiment analysis, and language identification.

If no Wav2Vec2 model is available for the specified language, word timestamps generated by Whisper will be used instead.

In [ ]:
if language in wav2vec2_langs:
    device = "cuda"
    alignment_model, metadata = whisperx.load_align_model(
        language_code=language, device=device
    )
    result_aligned = whisperx.align(
        whisper_results, alignment_model, metadata, vocal_target, device
    )
    word_timestamps = filter_missing_timestamps(
        result_aligned["word_segments"],
        initial_timestamp=whisper_results[0].get("start"),
        final_timestamp=whisper_results[-1].get("end"),
    )

    # clear gpu vram
    del alignment_model
    torch.cuda.empty_cache()
else:
    assert batch_size == 0, (  # TODO: add a better check for word timestamps existence
        f"Unsupported language: {language}, use --batch_size to 0"
        " to generate word timestamps using whisper directly and fix this error."
    )
    word_timestamps = []
    for segment in whisper_results:
        for word in segment["words"]:
            word_timestamps.append({"word": word[2], "start": word[0], "end": word[1]})

# 6. Using NeMo's MSDD model for speaker diarization
This code employs the NVIDIA NeMo MSDD (Multi-scale Diarization Decoder) model to perform speaker diarization on an audio signal. Speaker diarization is the process of separating an audio signal into different segments based on who is speaking at any given time.

In [ ]:
# Convert audio to mono for NeMo combatibility
sound = AudioSegment.from_file(vocal_target).set_channels(1)
ROOT = os.getcwd()
temp_path = os.path.join(ROOT, "temp_outputs")
os.makedirs(temp_path, exist_ok=True)
sound.export(os.path.join(temp_path, "mono_file.wav"), format="wav")

In [ ]:
# Initialize NeMo MSDD diarization model
msdd_model = NeuralDiarizer(cfg=create_config(temp_path)).to("cuda")
msdd_model.diarize()

del msdd_model
torch.cuda.empty_cache()

# 7. Mapping speakers to sentences according to timestamps

The code in this section reads the speaker labels and their corresponding timestamps from the output file generated by the NeMo MSDD model. It then uses the `get_words_speaker_mapping` function to map each word in the transcription to its respective speaker based on the timestamp information.

This mapping process ensures that each word is attributed to the correct speaker, creating a comprehensive representation of who spoke what and when. The resulting `wsm` (Word-Speaker Mapping) variable contains a list of dictionaries, where each dictionary represents a word and its associated speaker, start time, and end time.

By mapping speakers to sentences according to timestamps, the code lays the foundation for further analysis and processing of the diarized transcription.

In [ ]:
# Reading timestamps <> Speaker Labels mapping

speaker_ts = []
with open(os.path.join(temp_path, "pred_rttms", "mono_file.rttm"), "r") as f:
    lines = f.readlines()
    for line in lines:
        line_list = line.split(" ")
        s = int(float(line_list[5]) * 1000)
        e = s + int(float(line_list[8]) * 1000)
        speaker_ts.append([s, e, int(line_list[11].split("_")[-1])])

wsm = get_words_speaker_mapping(word_timestamps, speaker_ts, "start")

# 8. Enhancing speaker attribution with punctuation-based realignment

This section introduces a method for disambiguating speaker labels in cases where a sentence is split between two different speakers. By leveraging punctuation marks, the code determines the dominant speaker for each sentence in the transcription.

For example, consider the following scenario:

```
Speaker A: It's got to come from somewhere else. Yeah, that one's also fun because you know the lows are
Speaker B: going to suck, right? So it's actually it hits you on both sides.
```

If a sentence is split between two speakers, the code identifies the most frequent speaker label for each word in the sentence and assigns that label to the entire sentence. This approach helps improve the accuracy of speaker diarization, particularly when the Whisper model may not account for fine utterances like "hmm" and "yeah," but the NeMo Diarization Model does, resulting in inconsistent outputs.

Additionally, the code handles situations where one speaker is delivering a monologue while other speakers make occasional comments in the background. In such cases, it ignores the comments and assigns the entire monologue to the speaker who is speaking the majority of the time.

By realigning speech segments based on punctuation, the code provides a robust and reliable method for enhancing speaker attribution in the transcription.

In [ ]:
if language in punct_model_langs:
    # restoring punctuation in the transcript to help realign the sentences
    punct_model = PunctuationModel(model="kredor/punctuate-all")

    words_list = list(map(lambda x: x["word"], wsm))

    labled_words = punct_model.predict(words_list)

    ending_puncts = ".?!"
    model_puncts = ".,;:!?"

    # We don't want to punctuate U.S.A. with a period. Right?
    is_acronym = lambda x: re.fullmatch(r"\b(?:[a-zA-Z]\.){2,}", x)

    for word_dict, labeled_tuple in zip(wsm, labled_words):
        word = word_dict["word"]
        if (
            word
            and labeled_tuple[1] in ending_puncts
            and (word[-1] not in model_puncts or is_acronym(word))
        ):
            word += labeled_tuple[1]
            if word.endswith(".."):
                word = word.rstrip(".")
            word_dict["word"] = word

else:
    logging.warning(
        f"Punctuation restoration is not available for {language} language. Using the original punctuation."
    )

wsm = get_realigned_ws_mapping_with_punctuation(wsm)
ssm = get_sentences_speaker_mapping(wsm, speaker_ts)

# 9. Finalizing the diarization process

In this final section, the code performs essential cleanup tasks, exports the diarization results for further use, and replaces speaker IDs with their corresponding names. The main steps include:

1. **Saving the Speaker-Aware Transcript**: The `get_speaker_aware_transcript` function is used to generate a transcript that incorporates both the textual content and the corresponding speaker information. This transcript is then saved to a file with the same name as the input audio file, but with a ".txt" extension.

2. **Exporting the Diarization Results in SRT Format**: The `write_srt` function is employed to export the diarization results in the SubRip Text (SRT) format. This format is commonly used for subtitles and includes speaker labels and precise timestamps for each utterance. The SRT file is saved with the same name as the input audio file, but with a ".srt" extension.

3. **Cleaning Up Temporary Files**: The `cleanup` function is called to remove any temporary files or directories created during the diarization process. This step ensures a clean and organized working environment, freeing up storage space and maintaining system efficiency.

4. **Mapping Speaker IDs to Speaker Names**: The code reads the content of the previously saved speaker-aware transcript file and replaces the generic speaker IDs (e.g., "Speaker 0", "Speaker 1", "Speaker 2") with the actual names of the speakers.

By completing these final steps, the diarization process is concluded, and the results are made available for further analysis, post-processing, or integration with other tools and workflows. The exported speaker-aware transcript, SRT file, and the transcript with mapped speaker names provide valuable insights into the content and structure of the audio recording, enabling a wide range of applications, such as content analysis, speaker identification, and subtitle generation.

In [ ]:
with open(f"{os.path.splitext(audio_path)[0]}.txt", "w", encoding="utf-8-sig") as f:
    get_speaker_aware_transcript(ssm, f)

with open(f"{os.path.splitext(audio_path)[0]}.srt", "w", encoding="utf-8-sig") as srt:
    write_srt(ssm, srt)

cleanup(temp_path)

In [ ]:
# Open the file
with open(f"{os.path.splitext(audio_path)[0]}.txt", 'r') as f:
    text = f.read()

# Replace the speaker IDs with names
text = text.replace('Speaker 0','Ewa Jasiewicz')
text = text.replace('Speaker 1','Chris Faulkner')
text = text.replace('Speaker 2','Matt Frei')

# Write the file to disk
with open(audio_path[:-4] + '-with-speakers-names.txt', 'w') as f:
    f.write(text)